In [1]:
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
import os

# Model and API Key

In [2]:
# Load environment variables.
load_dotenv()

# Set the model name for our LLMs.
OPENAI_MODEL = "gpt-3.5-turbo"
# Store the API key in a variable.
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# Documents

In [4]:
# Additional imports for loading PDF documents and QA chain.
from langchain.document_loaders import PyPDFLoader
from langchain.chains.question_answering import load_qa_chain

In [5]:
# Create the PDF loader and load the document.
pdf_loader = PyPDFLoader("../Resources/Resume.pdf")

In [8]:
documents = pdf_loader.load()

In [9]:
# Initialize the model.
llm = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model_name=OPENAI_MODEL, temperature=0.0)

# Create the QA chain using the LLM.
chain = load_qa_chain(llm)

# Define a query as a string.
# query = "Could I write to Jane Doe in Spanish and expect her to understand?"
query = "What kinds of fields can I apply to?"

# Pass the documents and the query to the chain, and print the result.
result = chain.invoke({"input_documents" : documents, "question" : query})

In [10]:
result

{'input_documents': [Document(metadata={'source': '../Resources/Resume.pdf', 'page': 0}, page_content='Jane\nDoe\n123\nMaple\nAvenue\nAnytown,\nUSA\n(555)\n123-4567\njanedoe@email.com\nObjective:\nA\ndedicated\nprofessional\nseeking\na\nchallenging\nrole\nin\n\x00nance\nto\napply\nanalytical\nskills\nand\ndrive\nresults\nin\na\ndynamic\nenvironment.\nEducation:\nMaster\nof\nBusiness\nAdministration\n(MBA)\nUniversity\nof\nCityville,\nAnytown,\nUSA\n●\nGraduated\nwith\nhonors\nBachelor\nof\nScience\nin\nFinance\nState\nCollege,\nAnytown,\nUSA\n●\nGPA:\n3.8\nExperience:\nSenior\nFinancial\nAnalyst\nAlpha\nFinance\nSolutions,\nAnytown,\nUSA\n|\nJanuary\n2021\n-\nPresent\n●\nConducted\ncomprehensive\n\x00nancial\nanalysis,\nresulting\nin\na\n15%\nincrease\nin\ncost-eﬃciency\nwithin\nthe\n\x00rst\n\x00scal\nquarter.\n●\nDeveloped\ncomplex\n\x00nancial\nmodels\nand\nforecasts,\naiding\nstrategic\ndecision-making\nprocesses.\n●\nCollaborated\ncross-functionally\nto\noptimize\nbudget\nallocati

In [11]:
result['output_text']

'Based on your experience and skills in finance, you can apply to various fields such as investment banking, corporate finance, financial planning and analysis, asset management, consulting, risk management, and more. Your background in financial analysis, budgeting, forecasting, and data modeling can be valuable in a wide range of industries and roles within the finance sector.'

# Built-in Integrations

In [12]:
# Additional imports for loading Wikipedia content and QA chain.
from langchain_community.document_loaders import WikipediaLoader
from langchain.chains.question_answering import load_qa_chain

In [28]:
# Initialize the model.
llm = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model_name=OPENAI_MODEL, temperature=1.2)

# Define the wikipedia topic as a string.
wiki_topic = "climate change"

# Load the wikipedia results as documents, using a max of 2.
documents = WikipediaLoader(query=wiki_topic, load_max_docs=2, load_all_available_meta=True).load()

# Create the QA chain using the LLM.
chain = load_qa_chain(llm)

# Define a query as a string.
query = "How many more years will the earth last?"

# Pass the documents and the query to the chain, and print the result.
result = chain.invoke({"input_documents" : documents, "question" : query})

In [29]:
len(documents)

2

In [33]:
# documents[1].page_content

In [31]:
result['output_text']

"I don't know. The Earth's lifespan is not something that can be accurately predicted in terms of a specific number of years. Earth will likely continue to exist for billions of years, but the changes happening due to climate change affect life on Earth rather than the planet's existence itself."

# External APIs

In [27]:
# Additional imports for API chain.
from langchain.chains import APIChain

In [35]:
# Text description of API spec.
spec = """URL STRUCTURE
        Just hit http://numbersapi.com/number/type to get a plain text response, where
        type is one of trivia, math, date, or year. Defaults to trivia if omitted.
        number is an integer, or the keyword random, for which we will try to return
        a random available fact, or a day of year in the form month/day 
        (eg. 2/29, 1/09, 04/1), if type is date ranges of numbers"""

# Initialize the model.
llm = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model_name=OPENAI_MODEL, temperature=0.0)

# Create the API chain from the spec, using the LLM.
chain = APIChain.from_llm_and_api_docs(
    llm = llm,
    api_docs=spec,
    verbose=True,
    limit_to_domains=['http://numbersapi.com']
)

# Define a query as a dictionary.
query = {"question" : "Give me a funny fact about 07/15"}

# Run the chain using the query, and print the response.
result = chain.invoke(query)
print(result['output'])



> Entering new APIChain chain...
http://numbersapi.com/7/15/date
July 15th is the day in 1985 that the Nintendo Entertainment System, the best-selling game console of its time, is released in Japan.

> Finished chain.
On July 15th, 1985, the Nintendo Entertainment System was released in Japan.


In [36]:
# Additional imports for API chain and processing json.
from langchain.chains import APIChain
import json
import requests

In [37]:
# Store the API key in a variable.
api_key = os.getenv("NYT_API_KEY")

In [38]:
# Load the spec from json file and store the API key.
spec = requests.get("https://raw.githubusercontent.com/nytimes/public_api_specs/master/article_search/article_search_v2.json").json()
spec["api_key"] = api_key

In [40]:
type(spec)

dict

In [54]:
# Initialize the model.
model=ChatOpenAI(openai_api_key=OPENAI_API_KEY, model_name=OPENAI_MODEL, temperature=0.0)

# Create the API chain from the spec, using the LLM.
chain = APIChain.from_llm_and_api_docs(
    llm=model,
    api_docs=json.dumps(spec),
    verbose=False,
    limit_to_domains=["https://api.nytimes.com/"]
)
# Define a query as a string.
query = {"question" : "What are articles about Tracy Chapman's best songs?"}

# Run the chain using the query, and print the response.
result = chain.invoke(query)



> Entering new APIChain chain...
https://api.nytimes.com/svc/search/v2/articlesearch.json?q=Tracy%20Chapman%20best%20songs&fl=web_url,snippet,lead_paragraph,abstract,headline,pub_date,document_type,news_desk,byline,type_of_material&_id&api-key=eQjAU8uHfkI54uu1LzfhIGa2JH1MbYNe
{"status":"OK","copyright":"Copyright (c) 2024 The New York Times Company. All Rights Reserved.","response":{"docs":[{"abstract":"She is the first Black songwriter to receive the honor from the Country Music Awards. Her 1988 hit reached a new generation of fans as a cover by Luke Combs.","web_url":"https://www.nytimes.com/2023/11/09/arts/music/tracy-chapman-cma-awards-fast-car.html","snippet":"She is the first Black songwriter to receive the honor from the Country Music Awards. Her 1988 hit reached a new generation of fans as a cover by Luke Combs.","lead_paragraph":"Tracy Chapman won song of the year at the Country Music Awards on Wednesday for “Fast Car,” a folk ballad that topped the country charts more than 


> Finished chain.


In [55]:
result

{'question': "What are articles about Tracy Chapman's best songs?",
 'output': 'The response includes articles about Tracy Chapman\'s songs, such as her winning the Country Music Awards for "Fast Car," her return to the Grammy stage for a duet, and a profile of her in light of her album "New Beginning." It also mentions her past achievements at the Grammy Awards and her impact on pop music.'}

In [56]:
print(result['output'])

The response includes articles about Tracy Chapman's songs, such as her winning the Country Music Awards for "Fast Car," her return to the Grammy stage for a duet, and a profile of her in light of her album "New Beginning." It also mentions her past achievements at the Grammy Awards and her impact on pop music.
